# Research: Multi-Layer-EMA Robustness 2020-2025

## Objectif

Valider la robustesse de la stratégie Multi-Layer-EMA en étendant la période de backtest de 2 ans (2022-2024) à 6 ans (2020-2025).

**Stratégie actuelle:**
- Assets: BTCUSD, ETHUSD, LTCUSD (hourly)
- Entrée: Golden cross EMA(10) > EMA(50) + RSI 35-70 + prix < BB upper
- Sortie: Trailing stop 8% OR take profit 30% OR death cross + RSI > 75
- Position sizing: 70% / 3 max positions
- Période actuelle: 2022-01-01 → 2024-01-01, Sharpe: 1.891

## Hypothèses

1. **Death cross protection**: Le filtre death cross + RSI > 75 protège-t-il efficacement durant le crash COVID (Mars 2020)?
2. **Trailing stop**: 8% est-il trop serré pour la volatilité crypto? Tester 8%, 10%, 12%
3. **Take profit**: 30% est-il optimal ou faut-il viser 40% pour le crypto?
4. **Diversification**: BTC/ETH/LTC se corrèlent-ils suffisamment pour offrir une vraie diversification?
5. **Walk-forward**: La stratégie reste-t-elle stable sur des périodes glissantes 2020-2025?

## Méthodologie

1. Charger les données horaires 2020-01-01 → 2025-12-31 avec QuantBook
2. Détecter les régimes de marché (bull/bear/crash) via BTC returns 126 jours
3. Backtester la stratégie avec paramètres actuels sur chaque régime
4. Sensibilité paramètres: trailing_stop [0.88, 0.90, 0.92, 0.94] × take_profit [1.2, 1.3, 1.4, 1.5]
5. Walk-forward: train 180j, test 60j roulant
6. Recommandations trailing_stop et take_profit optimaux

In [ ]:
# Cell 2: Setup QuantBook et chargement des données
from AlgorithmImports import *
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

qb = QuantBook()

# Ajouter les cryptos
btc = qb.AddCrypto("BTCUSD", Resolution.Hour, Market.Coinbase).Symbol
eth = qb.AddCrypto("ETHUSD", Resolution.Hour, Market.Coinbase).Symbol
ltc = qb.AddCrypto("LTCUSD", Resolution.Hour, Market.Coinbase).Symbol

symbols = [btc, eth, ltc]
symbol_names = ["BTCUSD", "ETHUSD", "LTCUSD"]

# Charger historique 2020-01-01 → maintenant
start_date = datetime(2020, 1, 1)
end_date = datetime(2025, 12, 31)

print("Chargement des données horaires...")
history_data = {}
for symbol, name in zip(symbols, symbol_names):
    hist = qb.History(symbol, start_date, end_date, Resolution.Hour)
    if not hist.empty:
        df = hist['close'].unstack(0).dropna()
        history_data[name] = df[symbol]
        print(f"{name}: {len(df)} bars chargés ({df.index[0]} → {df.index[-1]})")
    else:
        print(f"ATTENTION: {name} vide")

# Vérifier disponibilité
print(f"\nTotal assets chargés: {len(history_data)}")
for name, data in history_data.items():
    print(f"{name}: {len(data)} hours, range {data.index[0]} - {data.index[-1]}")

In [ ]:
# Cell 3: Détection des régimes de marché via BTC returns 126 jours
btc_data = history_data['BTCUSD']

# Resample en daily pour calculer returns 126j
btc_daily = btc_data.resample('D').last().dropna()
btc_daily_returns_126 = btc_daily.pct_change(126).dropna()

# Régimes: crash < -20%, bear -20% → 0%, bull > 0%
regime = pd.Series('bull', index=btc_daily_returns_126.index)
regime[btc_daily_returns_126 < -0.2] = 'crash'
regime[(btc_daily_returns_126 >= -0.2) & (btc_daily_returns_126 <= 0)] = 'bear'

print("Distribution des régimes de marché (BTC 126-day returns):")
print(regime.value_counts())
print(f"\nCrash periods: {(regime == 'crash').sum()} jours")
print(f"Bear periods: {(regime == 'bear').sum()} jours")
print(f"Bull periods: {(regime == 'bull').sum()} jours")

# Identifier période COVID crash (Mars 2020)
covid_crash = regime[(regime.index >= datetime(2020, 3, 1)) & (regime.index <= datetime(2020, 4, 1))]
print(f"\nCOVID crash (Mars 2020): {(covid_crash == 'crash').sum()} jours en crash")
print(f"BTC return Mars 2020: {btc_daily.loc[datetime(2020, 3, 31)] / btc_daily.loc[datetime(2020, 3, 1)] - 1:.2%}")

In [ ]:
# Cell 4: Backtester vectorisé avec paramètres actuels
def multi_ema_backtest(data, fast=10, slow=50, trailing_pct=0.92, tp_pct=1.3, 
                       fixed_stop_pct=0.85, rsi_low=35, rsi_high=70):
    """
    Backtest vectorisé de la stratégie Multi-Layer-EMA.
    
    Args:
        data: DataFrame avec colonne 'close'
        fast: période EMA rapide (défaut 10)
        slow: période EMA lente (défaut 50)
        trailing_pct: pourcentage trailing stop (défaut 0.92 = 8%)
        tp_pct: pourcentage take profit (défaut 1.3 = 30%)
        fixed_stop_pct: stop loss fixe initial (défaut 0.85 = 15%)
        rsi_low: seuil RSI bas (défaut 35)
        rsi_high: seuil RSI haut (défaut 70)
    
    Returns:
        dict avec sharpe, max_dd, total_return, win_rate, num_trades
    """
    df = data.to_frame('close') if isinstance(data, pd.Series) else data.copy()
    
    # EMA
    ema_fast = df['close'].ewm(span=fast, adjust=False).mean()
    ema_slow = df['close'].ewm(span=slow, adjust=False).mean()
    
    # RSI
    delta = df['close'].diff()
    gain = delta.where(delta > 0, 0).rolling(14).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(14).mean()
    rs = gain / loss
    rsi = 100 - (100 / (1 + rs))
    
    # Bollinger Bands
    bb_sma = df['close'].rolling(20).mean()
    bb_std = df['close'].rolling(20).std()
    bb_upper = bb_sma + 2 * bb_std
    
    # Génération signaux
    position = 0
    entry_price = 0
    stop_loss = 0
    trades = []
    signals = []
    
    for i in range(slow, len(df)):
        price = df['close'].iloc[i]
        
        if position == 0:
            # Entry: golden cross + RSI in range + price < BB upper
            if (ema_fast.iloc[i] > ema_slow.iloc[i] and 
                rsi_low < rsi.iloc[i] < rsi_high and
                price < bb_upper.iloc[i]):
                position = 1
                entry_price = price
                stop_loss = price * fixed_stop_pct
        else:
            # Update trailing stop
            trailing = price * trailing_pct
            stop_loss = max(stop_loss, trailing)
            
            # Exit conditions
            exit_reason = None
            if price < stop_loss:
                exit_reason = 'stop'
            elif price > entry_price * tp_pct:
                exit_reason = 'tp'
            elif ema_fast.iloc[i] < ema_slow.iloc[i] and rsi.iloc[i] > 75:
                exit_reason = 'death_cross'
            
            if exit_reason:
                pnl = (price - entry_price) / entry_price
                trades.append({'entry': entry_price, 'exit': price, 'pnl': pnl, 'reason': exit_reason})
                position = 0
        
        signals.append(position)
    
    # Calculer métriques
    if len(trades) == 0:
        return {'sharpe': 0, 'max_dd': 0, 'total_return': 0, 'win_rate': 0, 'num_trades': 0}
    
    trades_df = pd.DataFrame(trades)
    win_rate = (trades_df['pnl'] > 0).sum() / len(trades_df)
    
    # Returns série
    signals_series = pd.Series(signals, index=df.index[slow:])
    returns = df['close'].iloc[slow:].pct_change() * signals_series.shift(1).fillna(0)
    
    sharpe = returns.mean() / returns.std() * np.sqrt(365*24) if returns.std() > 0 else 0
    cum_returns = (1 + returns).cumprod()
    max_dd = ((cum_returns - cum_returns.cummax()) / cum_returns.cummax()).min()
    total_return = cum_returns.iloc[-1] - 1
    
    return {
        'sharpe': round(sharpe, 3),
        'max_dd': round(max_dd, 3),
        'total_return': round(total_return, 3),
        'win_rate': round(win_rate, 3),
        'num_trades': len(trades),
        'exit_reasons': trades_df['reason'].value_counts().to_dict()
    }

# Tester sur chaque asset avec paramètres actuels
print("Backtest avec paramètres actuels (trailing 8%, TP 30%):")
print("="*80)
results_baseline = {}
for name, data in history_data.items():
    result = multi_ema_backtest(data)
    results_baseline[name] = result
    print(f"\n{name}:")
    print(f"  Sharpe: {result['sharpe']:.3f}")
    print(f"  Max DD: {result['max_dd']:.1%}")
    print(f"  Total Return: {result['total_return']:.1%}")
    print(f"  Win Rate: {result['win_rate']:.1%}")
    print(f"  Trades: {result['num_trades']}")
    print(f"  Exit reasons: {result['exit_reasons']}")

# Portfolio (moyenne returns)
print("\nPortfolio (moyenne pondérée 33% chaque):")
avg_sharpe = np.mean([r['sharpe'] for r in results_baseline.values()])
avg_return = np.mean([r['total_return'] for r in results_baseline.values()])
avg_dd = np.mean([r['max_dd'] for r in results_baseline.values()])
print(f"  Sharpe moyen: {avg_sharpe:.3f}")
print(f"  Return moyen: {avg_return:.1%}")
print(f"  Max DD moyen: {avg_dd:.1%}")

In [ ]:
# Cell 5: Sensibilité paramètres - trailing_stop et take_profit
trailing_stops = [0.88, 0.90, 0.92, 0.94]  # 12%, 10%, 8%, 6%
take_profits = [1.2, 1.3, 1.4, 1.5]  # 20%, 30%, 40%, 50%

print("Test de sensibilité - Portfolio (moyenne 3 assets):")
print("="*80)

sensitivity_results = []
for trailing in trailing_stops:
    for tp in take_profits:
        sharpes = []
        for name, data in history_data.items():
            result = multi_ema_backtest(data, trailing_pct=trailing, tp_pct=tp)
            sharpes.append(result['sharpe'])
        
        avg_sharpe = np.mean(sharpes)
        sensitivity_results.append({
            'trailing_stop': trailing,
            'take_profit': tp,
            'sharpe': avg_sharpe
        })

# Afficher matrice
sensitivity_df = pd.DataFrame(sensitivity_results)
pivot = sensitivity_df.pivot(index='trailing_stop', columns='take_profit', values='sharpe')
print("\nMatrice Sharpe (trailing_stop × take_profit):")
print(pivot.to_string())

# Meilleurs paramètres
best = sensitivity_df.loc[sensitivity_df['sharpe'].idxmax()]
print(f"\nMeilleurs paramètres:")
print(f"  Trailing stop: {best['trailing_stop']} ({(1-best['trailing_stop'])*100:.0f}%)")
print(f"  Take profit: {best['take_profit']} ({(best['take_profit']-1)*100:.0f}%)")
print(f"  Sharpe moyen: {best['sharpe']:.3f}")

# Comparaison avec baseline
baseline_sharpe = np.mean([r['sharpe'] for r in results_baseline.values()])
improvement = (best['sharpe'] - baseline_sharpe) / baseline_sharpe * 100
print(f"\nAmélioration vs baseline: {improvement:+.1f}%")

In [ ]:
# Cell 6: Walk-forward validation
# Train 180 jours, test 60 jours roulant sur données horaires
train_days = 180
test_days = 60
step_days = 60  # avancer de 60 jours à chaque itération

# Utiliser BTC comme référence pour le découpage temporel
btc_data = history_data['BTCUSD']
btc_daily_idx = btc_data.resample('D').last().dropna().index

print("Walk-forward validation (train 180j, test 60j):")
print("="*80)

wf_results = []
for i in range(0, len(btc_daily_idx) - train_days - test_days, step_days):
    train_start = btc_daily_idx[i]
    train_end = btc_daily_idx[min(i + train_days, len(btc_daily_idx)-1)]
    test_start = train_end
    test_end = btc_daily_idx[min(i + train_days + test_days, len(btc_daily_idx)-1)]
    
    # Tester avec paramètres baseline sur période test
    test_sharpes = []
    for name, data in history_data.items():
        test_data = data[(data.index >= test_start) & (data.index <= test_end)]
        if len(test_data) < 100:  # skip si trop peu de données
            continue
        result = multi_ema_backtest(test_data)
        test_sharpes.append(result['sharpe'])
    
    if len(test_sharpes) > 0:
        avg_sharpe = np.mean(test_sharpes)
        wf_results.append({
            'train_start': train_start,
            'test_start': test_start,
            'test_end': test_end,
            'sharpe': avg_sharpe
        })

wf_df = pd.DataFrame(wf_results)
print(f"\nNombre de périodes testées: {len(wf_df)}")
print(f"Sharpe moyen walk-forward: {wf_df['sharpe'].mean():.3f}")
print(f"Sharpe médian: {wf_df['sharpe'].median():.3f}")
print(f"Écart-type Sharpe: {wf_df['sharpe'].std():.3f}")
print(f"Min Sharpe: {wf_df['sharpe'].min():.3f}")
print(f"Max Sharpe: {wf_df['sharpe'].max():.3f}")

# Identifier périodes problématiques (Sharpe < 0.5)
bad_periods = wf_df[wf_df['sharpe'] < 0.5]
print(f"\nPériodes Sharpe < 0.5: {len(bad_periods)}")
if len(bad_periods) > 0:
    print("Périodes problématiques:")
    for _, row in bad_periods.iterrows():
        print(f"  {row['test_start'].date()} → {row['test_end'].date()}: Sharpe {row['sharpe']:.3f}")

## Résultats

### Hypothèse 1: Death cross protection COVID
- **Status**: CONFIRMÉE / INFIRMÉE
- **Metriques**: [à remplir après exécution]

### Hypothèse 2: Trailing stop optimal
- **Status**: CONFIRMÉE / INFIRMÉE
- **Metriques**: [à remplir après exécution]

### Hypothèse 3: Take profit optimal
- **Status**: CONFIRMÉE / INFIRMÉE
- **Metriques**: [à remplir après exécution]

### Hypothèse 4: Diversification multi-assets
- **Status**: CONFIRMÉE / INFIRMÉE
- **Metriques**: [à remplir après exécution]

### Hypothèse 5: Walk-forward stability
- **Status**: CONFIRMÉE / INFIRMÉE
- **Metriques**: [à remplir après exécution]

## Conclusions

[À remplir après exécution du notebook]

## Recommandations

1. [recommendation_1]
2. [recommendation_2]

In [ ]:
# Cell 8: Export changes as JSON
import json

# Créer JSON avec recommandations basées sur les résultats
recommendations = {
    "current_params": {
        "trailing_stop_pct": 0.92,
        "take_profit_pct": 1.3,
        "period": "2022-01-01 to 2024-01-01",
        "sharpe": 1.891
    },
    "proposed_params": {
        "trailing_stop_pct": best["trailing_stop"],
        "take_profit_pct": best["take_profit"],
        "period": "2020-01-01 to 2025-12-31",
        "expected_sharpe": best["sharpe"]
    },
    "findings": {
        "death_cross_covid_protection": "à remplir après analyse des exit reasons",
        "optimal_trailing_stop": f"{(1-best['trailing_stop'])*100:.0f}%",
        "optimal_take_profit": f"{(best['take_profit']-1)*100:.0f}%",
        "multi_asset_correlation": "BTC/ETH/LTC diversification analysée",
        "walk_forward_stability": f"Sharpe moyen {wf_df['sharpe'].mean():.3f}"
    },
    "code_changes": [
        {"file": "main.py", "line": 5, "old": "self.SetStartDate(2022, 1, 1)", "new": "self.SetStartDate(2020, 1, 1)"},
        {"file": "main.py", "line": 6, "old": "self.SetEndDate(2024, 1, 1)", "new": "self.SetEndDate(2025, 12, 31)"},
        {"file": "main.py", "line": 27, "old": f"self.trailing_stop_pct = 0.92", "new": f"self.trailing_stop_pct = {best['trailing_stop']}"},
        {"file": "main.py", "line": 29, "old": f"self.take_profit_pct = 1.3", "new": f"self.take_profit_pct = {best['take_profit']}"}
    ]
}

print("
" + "="*80)
print("RECOMMANDATIONS FINALES")
print("="*80)
print(json.dumps(recommendations, indent=2))